## 머신러닝 실습

### 텍스트 마이닝

#### 개요

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고, 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 이용
- 프로세스 - 텍스트 전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
- 텍스트 전처리 - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석

##### 특성 벡터화 / 추츨
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
-BoW, Word2ve 라이브러리(모듈) 존재

##### LDA
- 문서의 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_train.txt, ratings_test.txt 다운로드

- id(리뷰번호), document(리뷰), label(감성분류클래스 0은 부정, 1은 긍정 감성)

In [1]:
## 필요 라이브러리 사용 등록
import pandas as pd
import numpy as np
import re

In [2]:
# 모든 모듈/라이브러리는 __version__
np.__version__

'1.26.4'

In [3]:
## pandas 버전 상세정보
# pd.show_versions()

In [4]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [5]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [6]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
dfNsmcTrain.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [8]:
## 결측치 확인
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [9]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [10]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [11]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [12]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x:re.sub(r'[^가-힣|ㄱ-]+',' ',x))

In [13]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [14]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [15]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [16]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [17]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [19]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [20]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x:re.sub(r'[^가-힣|ㄱ-]+',' ',x))

In [21]:
dfNsmcTest

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [28]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] !=' ']

In [29]:
dfNsmcTrain

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1


In [30]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != ' ']

In [31]:
dfNsmcTest

,id,document,label
0,6270596,굳,1
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
5,7898805,음악이 주가 된 최고의 음악영화,1
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키지의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템등록정보 JAVA_HOME 설정
    - 프로그램 실행 : sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템 변수 > 새로 만들기 버튼 클릭
    - JAVA_HOME 과 경로 추가 등록
- JPype1 : 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [32]:
# Jpype1 설치
!pip install JPype1

   ---------------------------------------- 0.0/352.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/352.2 kB ? eta -:--:--
   ------------------- -------------------- 174.1/352.2 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 352.2/352.2 kB 3.1 MB/s eta 0:00:00


In [33]:
# koNLPy 설치
!pip install koNLPy 

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB 653.6 kB/s eta 0:00:30
   ---------------------------------------- 0.2/19.4 MB 2.0 MB/s eta 0:00:10
    --------------------------------------- 0.4/19.4 MB 3.1 MB/s eta 0:00:07
   - -------------------------------------- 0.6/19.4 MB 3.1 MB/s eta 0:00:07
   - -------------------------------------- 0.8/19.4 MB 3.3 MB/s eta 0:00:06
   - -------------------------------------- 1.0/19.4 MB 3.4 MB/s eta 0:00:06
   -- ------------------------------------- 1.2/19.4 MB 3.8 MB/s eta 0:00:05
   -- ------------------------------------- 1.4/19.4 MB 3.8 MB/s eta 0:00:05
   --- ------------------------------------ 1.7/19.4 MB 4.0 MB/s eta 0:00:05
   ---- ----------------------------------- 2.0/19.4 MB 4.2 MB/s eta 0:00:05
   ----- ---------------------------------- 2.5/19.4 MB 4.9 MB/s eta 0:00:04
   ----- --

In [34]:
import konlpy

In [35]:
konlpy.__version__

'0.6.0'

In [39]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
okt = Okt()

In [38]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [40]:
tfidf = TfidfVectorizer(tokenizer= oktTokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

In [41]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
nsmc_train_tfidf

<148434x113939 sparse matrix of type '<class 'numpy.float64'>'
	with 2681545 stored elements in Compressed Sparse Row format>

In [43]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)